In [1]:
# !sudo apt-get update
# !sudo apt-get install -y redis-server
# !sudo redis-server --daemonize yes
# !pip install redis

In [18]:
import json 
import re

In [19]:
path = "/home/ahmed/Desktop/AGH_Data_Agency_Holding_SA/Projet_LLM_Server/data_handling/quizzes_handling.json"
with open(path, 'r') as f: 
    data = json.load(f)

In [20]:
if isinstance(data, dict) and 'meals' in data:
    records_list = data['meals']
else:
    records_list = data

In [21]:
def clean_person_record(record):
    if not isinstance(record, dict):
        return str(record) if record is not None else ""
    
    combined_text = " ".join(str(record[f]) for f in record.keys() if record[f] not in ["N/A", None])
    combined_text = combined_text.replace("\n", " ")
    combined_text = re.sub(r"\s+", " ", combined_text).strip()
    return combined_text


In [22]:
texts = [clean_person_record(person) for person in records_list]

In [23]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="multi-qa-mpnet-base-dot-v1", device="cpu")

embedding_model.to("cpu")

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [24]:
from tqdm.auto import tqdm

meals_chunks = [{"sentence_chunk": text} for text in texts]
for item in tqdm(meals_chunks): 
    item["embedding"] = embedding_model.encode(item["sentence_chunk"], convert_to_tensor=True)

100%|██████████| 635/635 [01:37<00:00,  6.52it/s]


In [25]:
import torch

embeddings = [item["embedding"] for item in meals_chunks]
texts = [item["sentence_chunk"] for item in meals_chunks]
embeddings_tensor = torch.stack(embeddings)

In [26]:
torch.save({
            "embeddings": embeddings_tensor,
            "texts": texts
            }, 
"../data/quizzes_embeddings.pt")